<a href="https://colab.research.google.com/github/ann-las/DL_project/blob/main/metrics_with_confusion_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare metrics based on the DeepTMHMM confusion matrix



AUTHORS: Anna Schrøder Lassen (s173461), Astrid Brix Saksager (abrisa) &  Puck Quarles van Ufford (puqu)

DATE: December 2023

DESCRIPTION: This Colab calculates topology and protein type accuracies in the manner of the confusion matrix calculations done by the authors of the DeepTMHMM (Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609). Functions that are directly reused are cite this paper. The script calculate values for the predictions made by the DeepTMHMM for the test set partition used in our model, and for results from the end of our own training (for the training used to compute F1 micro values). Please insert paths to our_test.json, predictions_deeptmhmm.txt, micro_end_target.txt and micro_end_prediction.txt below.

In [3]:
our_test_file_path = '/content/drive/My Drive/DL/our_test.json' # Insert path for our_test.json here
deeptmhmm_file_path = '/content/drive/My Drive/DL/predictions_deeptmhmm.txt' # Insert path for predictions_deeptmhmm.txt here
target_data_path = "/content/drive/My Drive/DL/final_runs_predictions_targets/micro_end_target.txt" # Insert path for micro_end_target.txt here
prediction_data_path = "/content/drive/My Drive/DL/final_runs_predictions_targets/micro_end_prediction.txt" # Insert path for micro_end_prediction.txt

## 1) General preparations

Mount the drive:

In [4]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import packages:

In [5]:
import numpy as np
import sys
import torch
from typing import List, Union, Dict
from torch import tensor
import json

Topology metrics dictionary and functions:

In [6]:
LABELS: Dict[str,int] = {'I': 0, 'O':1, 'P': 2, 'S': 3, 'M':4, 'B': 5}

In [7]:
def array_to_string(one_hot_array, topology_dict):
    """
    Code from: Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609.
     """
    topology_string = ''
    for line in one_hot_array:
        top_key = np.argmax(line)
        topology_string += topology_dict[top_key]
    return topology_string


def label_list_to_topology(labels: Union[List[int], torch.Tensor]) -> List[torch.Tensor]:
    """
    Code from: Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609.

    Converts a list of per-position labels to a topology representation.
    This maps every sequence to list of where each new symbol start (the topology), e.g. AAABBBBCCC -> [(0,A),(3, B)(7,C)]

    Parameters
    ----------
    labels : list or torch.Tensor of ints
    List of labels.

    Returns
    -------
    list of torch.Tensor
    List of tensors that represents the topology.
    """

    if isinstance(labels, list):
        labels = torch.LongTensor(labels)

    if isinstance(labels, torch.Tensor):
        zero_tensor = torch.LongTensor([0])
    if labels.is_cuda:
        zero_tensor = zero_tensor.cuda()

    unique, count = torch.unique_consecutive(labels, return_counts=True)
    top_list = [torch.cat((zero_tensor, labels[0:1]))]
    prev_count = 0
    i = 0
    for _ in unique.split(1):
        if i == 0:
            i += 1
            continue
        prev_count += count[i - 1]
        top_list.append(torch.cat((prev_count.view(1), unique[i].view(1))))
        i += 1
    return top_list


def is_topologies_equal(topology_a, topology_b, minimum_seqment_overlap=5):
    """
    Code from: Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609.


    Checks whether two topologies are equal.
    E.g. [(0,A),(3, B)(7,C)]  is the same as [(0,A),(4, B)(7,C)]
    But not the same as [(0,A),(3, C)(7,B)]

    Parameters
    ----------
    topology_a : list of torch.Tensor
        First topology. See label_list_to_topology.
    topology_b : list of torch.Tensor
        Second topology. See label_list_to_topology.
    minimum_seqment_overlap : int
        Minimum overlap between two segments to be considered equal.

    Returns
    -------
    bool
        True if topologies are equal, False otherwise.
    """

    if isinstance(topology_a[0], torch.Tensor):
        topology_a = list([a.cpu().numpy() for a in topology_a])
    if isinstance(topology_b[0], torch.Tensor):
        topology_b = list([b.cpu().numpy() for b in topology_b])
    if len(topology_a) != len(topology_b):
        return False
    for idx, (_position_a, label_a) in enumerate(topology_a):
        if label_a != topology_b[idx][1]:
            if (label_a in (1,2) and topology_b[idx][1] in (1,2)): # assume O == P
                continue
            else:
                return False
        if label_a in (3, 4, 5):
            overlap_segment_start = max(topology_a[idx][0], topology_b[idx][0])
            overlap_segment_end = min(topology_a[idx + 1][0], topology_b[idx + 1][0])
            if label_a == 5:
                # Set minimum segment overlap to 3 for Beta regions
                minimum_seqment_overlap = 3
            if overlap_segment_end - overlap_segment_start < minimum_seqment_overlap:
                return False
    return True


def calculate_acc(correct, total):
    """
    Code from: Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609.
    """

    total = total.float()
    correct = correct.float()
    if total == 0.0:
        return 1
    return correct / total


def find_predicted_type(string_sequence):

  if "M" in string_sequence:
    if "S" in string_sequence:
      prot_type_string = "SP_TM"
      prot_type_int = 1

    elif "S" not in string_sequence:
      prot_type_string = "TM"
      prot_type_int = 0

  elif "B" in string_sequence:
    prot_type_string = "beta"
    prot_type_int = 4

  elif ("S" in string_sequence) and ("M" not in string_sequence) and ("B" not in string_sequence):
    prot_type_string = "SP_glob"
    prot_type_int = 2

  elif ("I" in string_sequence) or ("O" in string_sequence) or ("P" in string_sequence) and ("S" not in string_sequence) and ("M" not in string_sequence) and ("B" not in string_sequence):
    prot_type_string = "glob"
    prot_type_int = 3

  else:
    print("Could not define protein type for sequence:", string_sequence)

  return prot_type_string, prot_type_int



def calculate_type_correct_accuracy(confusion_matrix):
  """
  Code for calculations from: Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609.
  """

  # Calculate type correct ratio
  type_correct_ratio = \
      calculate_acc(confusion_matrix[0][0] + confusion_matrix[0][5], confusion_matrix[0].sum()) + \
      calculate_acc(confusion_matrix[1][1] + confusion_matrix[1][5], confusion_matrix[1].sum()) + \
      calculate_acc(confusion_matrix[2][2] + confusion_matrix[2][5], confusion_matrix[2].sum()) + \
      calculate_acc(confusion_matrix[3][3] + confusion_matrix[3][5], confusion_matrix[3].sum()) + \
      calculate_acc(confusion_matrix[4][4] + confusion_matrix[4][5], confusion_matrix[4].sum())
  type_accuracy = float((type_correct_ratio / 5).detach())

  return type_accuracy


def calculate_topology_accuracy(confusion_matrix):
  """
  Code for calculations from: Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609.
  """

  tm_accuracy = float(calculate_acc(confusion_matrix[0][5], confusion_matrix[0].sum()))
  sptm_accuracy = float(calculate_acc(confusion_matrix[1][5], confusion_matrix[1].sum()))
  sp_accuracy = float(calculate_acc(confusion_matrix[2][5], confusion_matrix[2].sum()))
  glob_accuracy = float(calculate_acc(confusion_matrix[3][5], confusion_matrix[3].sum()))
  beta_accuracy = float(calculate_acc(confusion_matrix[4][5], confusion_matrix[4].sum()))

  return tm_accuracy, sptm_accuracy, sp_accuracy, glob_accuracy, beta_accuracy


def calculate_type_accuracy(confusion_matrix):
  """
  Code for calculations from: Hallgren J, Tsirigos KD, Pedersen MD, et al. DeepTMHMM predicts alpha and beta transmembrane proteins using deep neural networks. bioRxiv; 2022. DOI: 10.1101/2022.04.08.487609.
  """

  tm_type_acc = float(calculate_acc(confusion_matrix[0][0] + confusion_matrix[0][5], confusion_matrix[0].sum()))
  tm_sp_type_acc = float(calculate_acc(confusion_matrix[1][1] + confusion_matrix[1][5], confusion_matrix[1].sum()))
  sp_type_acc = float(calculate_acc(confusion_matrix[2][2] + confusion_matrix[2][5], confusion_matrix[2].sum()))
  glob_type_acc = float(calculate_acc(confusion_matrix[3][3] + confusion_matrix[3][5], confusion_matrix[3].sum()))
  beta_type_acc = float(calculate_acc(confusion_matrix[4][4] + confusion_matrix[4][5], confusion_matrix[4].sum()))

  return tm_type_acc, tm_sp_type_acc, sp_type_acc, glob_type_acc, beta_type_acc


## 2) DeepTMHMM results

Get uniprots for the partition of data that we use for test:

In [9]:
# Define our uniprots (test)
#our_test_file = open('/content/drive/My Drive/DL/our_test.json')
our_test_file = open(our_test_file_path)
test_dict = json.load(our_test_file)
our_test_file.close()

our_uniprots = list(test_dict.keys())
#print(our_uniprots)
print(len(our_uniprots))

714


Get DeepTMHMM predictions:

In [10]:
# Name of the deepTMHMM file
#deeptmhmm_file = '/content/drive/My Drive/DL/predictions_deeptmhmm.txt'
deeptmhmm_file = deeptmhmm_file_path

# Dict of labels
#LABELS: Dict[str,int] = {'I': 0, 'O':1, 'P': 2, 'S': 3, 'M':4, 'B': 5}
#labels_trans = str.maketrans("IOPSMB","012345")

Get lists of ground truth and DeepTMHMM prediction sequences:

In [11]:
# Lists of sequences
deep_tmhmmm_uniprots_our_uniprots = list()
deep_tmhmmm_uniprots_not_our_uniprots = list()
sequences = list()
ground_truth = list()
tmhmm_prediction = list()

read_lines = False
count = 0

# Run through file and extract sequences
with open(deeptmhmm_file, 'r') as file:
    for line in file:

        # Initiate reading at correct sequences
        if line.startswith('>'):
            protein_uniprot = line[1:-1]

            if protein_uniprot in our_uniprots:
                read_lines = True
                deep_tmhmmm_uniprots_our_uniprots.append(protein_uniprot)

            else:
                read_lines = False
                deep_tmhmmm_uniprots_not_our_uniprots.append(protein_uniprot)


        if not line.startswith('>') and read_lines == True and count == 0:
            sequences.append(line[:-1])
            count += 1

        elif not line.startswith('>') and read_lines == True and count == 1:
            ground_truth.append(line[:-1])
            count += 1

        elif not line.startswith('>') and read_lines == True and count == 2:
            tmhmm_prediction.append(line[:-1])
            count = 0
            readlines = False

Check results:

In [12]:
# Number of uniprots
print(len(our_uniprots))
print(len(deep_tmhmmm_uniprots_our_uniprots))
print(len(deep_tmhmmm_uniprots_not_our_uniprots))

# Lenghts of sequences
print(len(sequences))
print(len(ground_truth))
print(len(tmhmm_prediction))

# Print sequences
#print(ground_truth)
#print(tmhmm_prediction)

714
714
2860
714
714
714


Get target labels:

In [13]:
target_list = ground_truth
prediction_list = tmhmm_prediction


# Get target labels
target_labels = list()
for string in target_list:
  target_label_list = list()
  for letter in string:
    target_label_list.append(LABELS[letter])

  target_labels.append(target_label_list)

# Get predicted labels
predicted_labels = list()
for string in prediction_list:
  prediction_label_list = list()
  for letter in string:
    prediction_label_list.append(LABELS[letter])

  predicted_labels.append(prediction_label_list)

Run through lists to get predictions:

In [14]:
predicted_types_string = list()
predicted_types_int = list()
target_types_string = list()
target_types_int = list()

for i in range(len(prediction_list)):
  predicted_types_string.append(find_predicted_type(prediction_list[i])[0])
  predicted_types_int.append(find_predicted_type(prediction_list[i])[1])

  target_types_string.append(find_predicted_type(target_list[i])[0])
  target_types_int.append(find_predicted_type(target_list[i])[1])

#print(predicted_types_string)
#print(predicted_types_int)
#print(target_types_int)

Initiate confusion matrix:

In [15]:
# Create confusion matrix
confusion_matrix = torch.zeros((6,6), dtype = torch.int64)
print(confusion_matrix)

tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]])


Fill confusion matrix:

In [16]:
# Fill out confusion matrix

for i in range(len(target_types_int)):

  # Get types of protein
  target_type = target_types_int[i]
  predicted_type = predicted_types_int[i]

  # Get topologies of protein
  #target_topology = target_topologies[i]
  #predicted_topology = predicted_topologies[i]
  target_topology = label_list_to_topology(target_labels[i])
  predicted_topology = label_list_to_topology(predicted_labels[i])

  # Check topology match
  prediction_topology_match = is_topologies_equal(target_topology, predicted_topology, 5)

  # Fill confusion matrix
  if target_type == predicted_type:
    # if we guessed the type right for SP+GLOB or GLOB, count the topology as correct
    if target_type == 2 or target_type == 3 or prediction_topology_match:
      confusion_matrix[target_type][5] += 1

    else:
      confusion_matrix[target_type][predicted_type] += 1

  else:
    confusion_matrix[target_type][predicted_type] += 1



print(confusion_matrix)

tensor([[  7,   1,   0,   1,   0,  68],
        [  0,   0,   0,   0,   0,  21],
        [  0,   3,   0,   3,   1, 193],
        [  7,   0,   1,   0,   0, 392],
        [  0,   0,   0,   0,   5,  11],
        [  0,   0,   0,   0,   0,   0]])


Print accuracies:

In [17]:
print("Type correct accuracy:")
print(calculate_type_correct_accuracy(confusion_matrix))
print(" ")

print("Topology accuracies for protein types:")
top_accuracies = calculate_topology_accuracy(confusion_matrix)
print('tm_accuracy:', top_accuracies[0])
print('sptm_accuracy:', top_accuracies[1])
print('sp_accuracy:', top_accuracies[2])
print('glob_accuracy:', top_accuracies[3])
print('beta_accuracy:', top_accuracies[4])
print(" ")

print("Type accuracies for protein types:")
type_accuracies = calculate_type_accuracy(confusion_matrix)
print("tm_type_acc:", type_accuracies[0])
print("tm_sp_type_acc:", type_accuracies[1])
print("sp_type_acc:", type_accuracies[2])
print("glob_type_acc:", type_accuracies[3])
print("beta_type_acc:", type_accuracies[4])

Type correct accuracy:
0.9838051795959473
 
Topology accuracies for protein types:
tm_accuracy: 0.8831169009208679
sptm_accuracy: 1.0
sp_accuracy: 0.9649999737739563
glob_accuracy: 0.9800000190734863
beta_accuracy: 0.6875
 
Type accuracies for protein types:
tm_type_acc: 0.9740259647369385
tm_sp_type_acc: 1.0
sp_type_acc: 0.9649999737739563
glob_type_acc: 0.9800000190734863
beta_type_acc: 1.0


## 3) Our results

Particular dictionary for the model (NB: Important to insert!)

In [29]:
### Get topology dict ###
#site_to_num: Dict[str,int] = {'M': 0, 'S': 1, 'P': 2, 'B': 3, 'O': 4, 'I': 5} #micro_best #### ---- IMPORTANT: INSERT HERE FOR YOUR TEST ---- ####
#site_to_num: Dict[str,int] =  {'O': 0, 'P': 1, 'S': 2, 'M': 3, 'B': 4, 'I': 5} #macro_best
site_to_num: Dict[str, int] = {'M': 0, 'I': 1, 'O': 2, 'S': 3, 'P': 4, 'B': 5} # micro_end
#site_to_num: Dict[str, int] = {'P': 0, 'M': 1, 'B': 2, 'O': 3, 'I': 4, 'S': 5} # macro_end
num_to_site: Dict[int,str] = {value: key for key, value in site_to_num.items()}

Load target and prediction arrays:

In [30]:
#### Open target file  ####
#filename = "/content/drive/My Drive/DL/target_Felix_metrics_test.txt"
#filename = "/content/drive/My Drive/DL/final_runs_predictions_targets/micro_end_target.txt"
filename = target_data_path
firstarray = True

with open(filename, 'r') as file:
    # Initialize an empty list to store the loaded arrays
    target_loaded_arrays = []

    for line in file:
        if line.startswith("#NewArray"):

            if firstarray == True:
                firstarray = False
                current_array_data = []

            elif firstarray == False:
                target_loaded_arrays.append(np.array(current_array_data))
                current_array_data = []
        else:
            current_array_data.append(np.fromstring(line, sep='\t'))

    target_loaded_arrays.append(np.array(current_array_data))


# Print the loaded arrays
#for i, arr in enumerate(target_loaded_arrays):
#    print(f"Array {i + 1}:\n{arr}")



#### Open prediction file  ####
#filename = "/content/drive/My Drive/DL/prediction_Felix_metrics_test.txt"
#filename = "/content/drive/My Drive/DL/final_runs_predictions_targets/macro_end_prediction.txt"
filename = prediction_data_path
firstarray = True

with open(filename, 'r') as file:
    # Initialize an empty list to store the loaded arrays
    pred_loaded_arrays = []

    for line in file:
        if line.startswith("#NewArray"):

            if firstarray == True:
                firstarray = False
                current_array_data = []

            elif firstarray == False:
                pred_loaded_arrays.append(np.array(current_array_data))
                current_array_data = []
        else:
            line = np.fromstring(line, sep='\t')
            line_as_int = (line == np.max(line)).astype(int)
            current_array_data.append(line_as_int)

    pred_loaded_arrays.append(np.array(current_array_data))


# Print the loaded arrays
#for i, arr in enumerate(pred_loaded_arrays):
#    print(f"Array {i + 1}:\n{arr}")


# Check result
if len(target_loaded_arrays) == len(pred_loaded_arrays):
    print('Same number of arrays.')
else:
    print('Number of prediction and target arrays do not match.')
    sys.exit(1)

Same number of arrays.


Create lists of sequences:

In [31]:
# Create lists of sequences
prediction_list = list()
target_list = list()

for i in range(len(target_loaded_arrays)):

    prediction_sequence = array_to_string(pred_loaded_arrays[i], num_to_site)
    prediction_list.append(prediction_sequence)

    target_sequence = array_to_string(target_loaded_arrays[i], num_to_site)
    target_list.append(target_sequence)

print(len(prediction_list))
print(len(target_list))
#print(prediction_list)
#print(target_list)

711
711


Get target labels:


In [32]:
# Get target labels
target_labels = list()
for string in target_list:
  target_label_list = list()
  for letter in string:
    target_label_list.append(LABELS[letter])
    #print(target_label_list)
    #if "0" in target_label_list:
    #  print("yes")

  target_labels.append(target_label_list)

# Get predicted labels
predicted_labels = list()
for string in prediction_list:
  prediction_label_list = list()
  for letter in string:
    prediction_label_list.append(LABELS[letter])

  predicted_labels.append(prediction_label_list)


Run through lists to get predictions:

In [33]:
predicted_types_string = list()
predicted_types_int = list()
target_types_string = list()
target_types_int = list()

for i in range(len(prediction_list)):
  predicted_types_string.append(find_predicted_type(prediction_list[i])[0])
  predicted_types_int.append(find_predicted_type(prediction_list[i])[1])

  target_types_string.append(find_predicted_type(target_list[i])[0])
  target_types_int.append(find_predicted_type(target_list[i])[1])

  #if "B" in target_list[i]:
  #  print(target_types_string[i], target_list[i])
  #if target_types_string[i] == "SP_glob":
  #  print('yay', target_list[i])


#print(predicted_types_string)
#print(predicted_types_int)
#print(target_types_int)

Create confusion matrix:

In [34]:
# Create confusion matrix
confusion_matrix = torch.zeros((6,6), dtype = torch.int64)
print(confusion_matrix)

tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]])


Fill out confusion matrix:

In [35]:
# Fill out confusion matrix

for i in range(len(target_types_int)):

  # Get types of protein
  target_type = target_types_int[i]
  predicted_type = predicted_types_int[i]

  # Get topologies of protein
  #target_topology = target_topologies[i]
  #predicted_topology = predicted_topologies[i]
  target_topology = label_list_to_topology(target_labels[i])
  predicted_topology = label_list_to_topology(predicted_labels[i])

  # Check topology match
  prediction_topology_match = is_topologies_equal(target_topology, predicted_topology, 5)

  # Fill confusion matrix
  if target_type == predicted_type:
    # if we guessed the type right for SP+GLOB or GLOB, count the topology as correct
    if target_type == 2 or target_type == 3 or prediction_topology_match:
      confusion_matrix[target_type][5] += 1

    else:
      confusion_matrix[target_type][predicted_type] += 1

  else:
    confusion_matrix[target_type][predicted_type] += 1



print(confusion_matrix)

tensor([[  0,   0,   0,   0,   0,   0],
        [ 74,   0,   0,  23,   0,   0],
        [311,   0,   0,  87,   0,   0],
        [  0,   0,   0,   0,   0,   0],
        [194,   0,   0,  22,   0,   0],
        [  0,   0,   0,   0,   0,   0]])


Print accuracies

In [36]:
print("Type correct accuracy:")
print(calculate_type_correct_accuracy(confusion_matrix))
print(" ")

print("Topology accuracies for protein types:")
top_accuracies = calculate_topology_accuracy(confusion_matrix)
print('tm_accuracy:', top_accuracies[0])
print('sptm_accuracy:', top_accuracies[1])
print('sp_accuracy:', top_accuracies[2])
print('glob_accuracy:', top_accuracies[3])
print('beta_accuracy:', top_accuracies[4])
print(" ")

print("Type accuracies for protein types:")
type_accuracies = calculate_type_accuracy(confusion_matrix)
print("tm_type_acc:", type_accuracies[0])
print("tm_sp_type_acc:", type_accuracies[1])
print("sp_type_acc:", type_accuracies[2])
print("glob_type_acc:", type_accuracies[3])
print("beta_type_acc:", type_accuracies[4])

Type correct accuracy:
0.4000000059604645
 
Topology accuracies for protein types:
tm_accuracy: 1.0
sptm_accuracy: 0.0
sp_accuracy: 0.0
glob_accuracy: 1.0
beta_accuracy: 0.0
 
Type accuracies for protein types:
tm_type_acc: 1.0
tm_sp_type_acc: 0.0
sp_type_acc: 0.0
glob_type_acc: 1.0
beta_type_acc: 0.0
